# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load data from WeatherPy export
csv_file = "../output_data/cities.csv"
cities = pd.read_csv(csv_file)

In [3]:
# Preview data
cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kralendijk,12.1500,-68.2667,85.98,70.0,40.0,19.57,BQ,1.628174e+09
1,barrow,71.2906,-156.7887,46.42,95.0,90.0,12.66,US,1.628175e+09
2,ushuaia,-54.8000,-68.3000,49.62,61.0,0.0,4.61,AR,1.628174e+09
3,kodiak,57.7900,-152.4072,53.11,94.0,75.0,0.00,US,1.628175e+09
4,flinders,-34.5833,150.8552,51.80,74.0,1.0,8.61,AU,1.628175e+09
...,...,...,...,...,...,...,...,...,...
552,sistranda,63.7256,8.8340,69.76,68.0,91.0,9.93,NO,1.628175e+09
553,bainbridge,47.6262,-122.5212,68.47,80.0,75.0,0.00,US,1.628175e+09
554,mahina,-17.5065,-149.4890,74.97,78.0,20.0,5.75,PF,1.628175e+09
555,tshikapa,-6.4167,20.8000,99.19,13.0,0.0,3.36,CD,1.628175e+09


### Humidity Heatmap

In [4]:
# Create heatmap with cities and humidity
gmaps.configure(api_key=g_key)
locations = cities[['Lat','Lng']]
humidity = cities['Humidity']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Set ideal weather criteria
max_temp = 80
min_temp = 70
wind = 5
cloudiness = 10
humidity = 65

# Filter cities by criteria, create copy, and drop null values
filtered_cities = cities.loc[(cities['Max Temp']>min_temp) & (cities['Max Temp']<max_temp) & (cities['Cloudiness']<cloudiness)& (cities['Wind Speed']<wind) & (cities['Humidity']<humidity),:]
ideal_cities = filtered_cities.copy()
ideal_cities.dropna(inplace=True)

# Display results
ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
177,griffith,41.5284,-87.4237,79.90,58.0,1.0,1.99,US,1.628175e+09
199,masvingo,-20.0637,30.8277,72.12,23.0,0.0,3.67,ZW,1.628175e+09
268,caranavi,-15.8167,-67.5500,74.03,41.0,0.0,3.65,BO,1.628175e+09
297,vallenar,-28.5708,-70.7581,70.70,42.0,0.0,2.35,CL,1.628175e+09
314,corrales,35.2378,-106.6067,79.97,59.0,1.0,1.99,US,1.628175e+09
348,aguilares,-27.4338,-65.6143,73.44,29.0,4.0,4.16,AR,1.628175e+09
505,giyani,-23.3025,30.7187,79.16,24.0,0.0,4.90,ZA,1.628175e+09


### Hotel Map

In [6]:
# Create new dataframe for hotel data
hotel_df = ideal_cities[['City','Country','Lat','Lng']].copy()

In [7]:
# Search for hotels near each city coordinate, store first hotel for each city into dataframe

params = {"radius":5000,
          "types":"lodging",
          "keyword":"hotel",
          "key":g_key}

for index, row in ideal_cities.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'

    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        response = requests.get(base_url, params = params).json()
        hotel_df.loc[index, 'Hotel Name']=response['results'][0]['name']

# Remove row from dataframe if nearby hotel is not found
    except:
        hotel_df.drop(index, inplace = True)

In [8]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
177,griffith,US,41.5284,-87.4237,Best Western Crossroads Inn
199,masvingo,ZW,-20.0637,30.8277,Regency Flamboyant Hotel
268,caranavi,BO,-15.8167,-67.5500,Hotel Wara
297,vallenar,CL,-28.5708,-70.7581,Hotel Puerto de Vega
314,corrales,US,35.2378,-106.6067,Days Inn by Wyndham Rio Rancho
348,aguilares,AR,-27.4338,-65.6143,Posada y Eventos La Casona
505,giyani,ZA,-23.3025,30.7187,Accommodation in Giyani - Vahlavi Lodge


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Create markers for hotel info and play layer on map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))